## AI Agent Stocks Insights and News

### Overview
Carlos is a small investor who constantly needs to stay informed about the latest market trends and financial news. Due to his demanding schedule, Carlos finds it challenging and inefficient to manually sift through numerous articles, reports, and data sources to get accurate and timely insights.

To address this, Carlos uses the AI Agent Stocks Insights, an intelligent solution powered by generative AI, enabling him to quickly and efficiently access real-time information tailored to his financial research needs.

### Problem Statement
As a busy professional balancing data science projects and investment decisions, Carlos requires quick, reliable, and concise information about company financial standings, stock prices, and relevant market news. His current methods are too time-consuming and inefficient, leading to delays and potential missed opportunities.

### Solution Approach
Here's how Carlos leverages AI Agent Stocks Insights:

Financial Analysis:
When Carlos needs information about NVIDIA's financial performance, he simply asks, "Give me a company overview of Nvidia Corporation." The AI Agent retrieves and synthesizes comprehensive company data, presenting it clearly and succinctly.
To get an instant update on NVIDIA's stock price, Carlos prompts, "Give me the current stock price for NVDA," and immediately receives accurate, up-to-date pricing information.

News Insights:
To stay ahead of emerging trends, Carlos queries, "What is the latest news on AI?" The AI Agent promptly fetches the most recent and relevant articles, providing Carlos with concise summaries and essential insights.
This notebook demonstrates how Carlos efficiently interacts with the AI Agent using generative AI capabilities, seamlessly connecting with financial and news APIs, significantly enhancing his productivity and investment decision-making process.

### 1. Setup
Start by installing and importing the Python SDK.

In [8]:
from google import genai
from google.genai import types
import requests
import os
from dotenv import load_dotenv
import grpc

# Try to import grpc_status, set to None if not available
try:
    import grpc_status
except ImportError:
    grpc_status = None

from IPython.display import Markdown


#### Automated retry
Define a retry policy. The model might make multiple consecutive calls automatically for a complex query, this ensures the client retries if it hits quota limits.

In [9]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

### 2. Define your helper functions
Implement each function in Python to fetch data from external services

In [10]:
import requests

BASE_ALPHA = "https://www.alphavantage.co/query"

def get_stock_price(symbol: str) -> dict:
    """Fetch the current stock price of a given company in dollar"""
    params = {
        "function": "GLOBAL_QUOTE",
        "symbol": symbol,
        "apikey": ALPHA_KEY
    }
    data = requests.get(BASE_ALPHA, params=params, timeout=10).json()
    quote = data.get("Global Quote", {})

    return {
        "symbol": quote.get("01. symbol"),
        "price": float(quote.get("05. price", 0)),
        "date": quote.get("07. latest trading day")
    }

def get_company_overview(symbol: str) -> dict:
    """Get company details and other financial data"""
    params = {"function": "OVERVIEW", "symbol": symbol, "apikey": ALPHA_KEY}
    ov = requests.get(BASE_ALPHA, params=params, timeout=10).json()
    fields = ["Name","Description","Sector","MarketCapitalization",
              "PERatio","DividendYield","52WeekHigh","52WeekLow"]

    return {k: ov.get(k, "NA") for k in fields}


def get_news(topic: str, n:int = 5) -> dict:
    """Get the latest news headlines for a given company."""
    url = ("https://newsapi.org/v2/everything?"
           f"q={topic}&language=en&sortBy=publishedAt&pageSize={n}&apiKey={NEWS_KEY}")
    articles = requests.get(url, timeout=10).json().get("articles", [])
    news = [{"title": a["title"],
             "url": a["url"],
             "published": a["publishedAt"][:10]} for a in articles]

    return {"topic": topic, "count": len(news), "articles": news}

### 3. Define Function Schemas and parameter descriptions
We define three primary function schemas: get_company_overview, get_stock_price, and get_news:

In [11]:
import google.generativeai as genai

genai.configure(api_key=GOOGLE_KEY)

# The tools argument should be a list of Tool objects.
# Each Tool object should have a list of function_declarations.
tools_stocks = genai.types.Tool(
    function_declarations=[
        {
            "name": "get_stock_price",
            "description": "Return the latest USD trading price for a stock symbol (e.g. GOOG).",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {"type": "string", "description": "Ticker (upper‑case)"},
                },
                "required": ["symbol"]
            }
        },
        {
            "name": "get_company_overview",
            "description": "Return a concise fundamental overview for a public company.",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {"type": "string", "description": "Ticker (upper‑case)"},
                },
                "required": ["symbol"]
            }
        },
        {
            "name": "get_news",
            "description": "Return recent news articles on a topic or company.",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {"type": "string", "description": "Search query"},
                    "n": {"type": "integer", "description": "Max number of articles"},
                },
                "required": ["topic"]
            }
        }
    ]
)

### 4. Initialize model
Initialize the Gemini model with the desired model parameters and Tool that we defined earlier

In [12]:
SYS = (
"""
You are a seasoned Finance Coach and AI‑powered Investment Advisor. 
Your role is to deliver up‑to‑the‑minute market trends, breaking financial news and tailored insights. 
Whenever needed, you fetch real‑time data—stock quotes, company overviews or relevant news articles—and synthesize it into clear, concise recommendations. 
Always maintain a professional tone and help the user make fast, well‑informed financial decisions.
"""
)

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    tools= [tools_stocks], # tools_stocks as a list
    system_instruction=SYS,
    
)

chat = model.start_chat()

### 4. Function to execute Gemini Function Calling
In this step, we wire up the conversation loop using the Gemini SDK. The AI Agent will detect when to call our helper functions and handle the responses automatically.

In [13]:
def run_agent(user_prompt: str):
    response = chat.send_message(user_prompt, stream=False)
    part = response.candidates[0].content.parts[0]

    if hasattr(part, "function_call"):
        fn_call = part.function_call
        fn_name = fn_call.name
        args = fn_call.args or {}

        # Check if the function name is valid and not empty
        if fn_name and fn_name in globals():
            # Executa a função Python correspondente
            result = globals()[fn_name](**args)
            # Send the result back to the model
            response = chat.send_message(content=str(result), stream=False)  
        else:
            print(f"Function '{fn_name}' not found or invalid.")  # Inform the user about the issue
            return "I couldn't find the information you requested." # Provide a fallback response

    # Check if response is a string before accessing .text
    if isinstance(response, str):
        return response
    else:
        # Access the text content directly from the response
        return response.text

### 5. Output
Sample prompts

In [15]:
prompt = "What's the latest news about AI"
response = run_agent(prompt)
Markdown(response)

NameError: name 'rpc_status' is not defined

In [ ]:
# Prompt related to stock price
prompt = "What is the current stock price for Google?"
response = run_agent(prompt)
print(response)

In [ ]:
prompt = "Give me a company overview of Tesla Company"
response = run_agent(prompt)
Markdown(response)